# Question 3

## Review question 1 and 2 

In [68]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
wikipedia_html = res.text

soup = BeautifulSoup(wikipedia_html, "html.parser")
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [69]:
import pandas as pd
df=pd.DataFrame(table_contents)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [70]:
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_geo = pd.read_csv(csv_path)

df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [71]:
df = df.sort_values(by="PostalCode", ignore_index=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [72]:
# Obtain latitude and longitude coordinates
latitude=df_geo["Latitude"]
longitude = df_geo["Longitude"]

#Merge data
df["Latitude"] = latitude
df["Longitude"] = longitude

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Install folium

In [75]:
! pip install folium==0.5.0
import folium 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [76]:
toronto_df = df[df['Borough'].isin(['Central Toronto', 'Downtown Toronto','West Toronto', 'East YorkEast Toronto'])].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
4,M4S,Central Toronto,Davisville,43.704324,-79.388790


## Create a map of Toronto

In [77]:
toronto_latitude = 43.651070
toronto_longitude = -79.347015

map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare credentials

In [78]:
CLIENT_ID = 'KP23O2U4XKDE12JWQ52IANQIJUO01POWJIVTAHI0WGC5IX50'      # your Foursquare ID
CLIENT_SECRET = 'FWF31DOXLUP3TWZVQRAVFIRPXG5SCFXK2U3HSHUIEW1HQAGH'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KP23O2U4XKDE12JWQ52IANQIJUO01POWJIVTAHI0WGC5IX50
CLIENT_SECRET:FWF31DOXLUP3TWZVQRAVFIRPXG5SCFXK2U3HSHUIEW1HQAGH


## Get Venues nearby 

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Danforth  East
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea


In [81]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
1,The Danforth East,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store
2,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
3,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
4,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line


In [84]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 207 uniques categories.


## Create the Dataframe of Neighborhood Venues

In [85]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot['neighborhood'] = toronto_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Danforth East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Danforth East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
toronto_grouped = toronto_onehot.groupby('neighborhood').mean().reset_index()
toronto_grouped.head()

,neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.043478,0.0,0.000000,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455,0.000000,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.071429,0.071429,0.142857,0.142857,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.015152,0.0,0.000000,0.015152,0.0,0.015152
4,Christie,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000


In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
import numpy as np

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = toronto_grouped['neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Bakery,Sandwich Place,Coffee Shop,Vegetarian / Vegan Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Park,Indian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Sandwich Place,Breakfast Spot,Bakery,Furniture / Home Store,Intersection,Italian Restaurant,Japanese Restaurant,Convenience Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Café,Japanese Restaurant,Bank,Burger Joint,Restaurant,Pizza Place
4,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Coffee Shop,Nightclub,Restaurant,Eastern European Restaurant,Donut Shop


## Now Cluster Neighborhoods

In [90]:
from sklearn.cluster import KMeans

#set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [93]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,1,Park,Convenience Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Park,Department Store,Gym,Breakfast Spot,Playground,Sandwich Place,Food & Drink Shop,Donut Shop,Doner Restaurant
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Yoga Studio,Bagel Shop,Clothing Store,Chinese Restaurant,Café,Diner,Fast Food Restaurant,Spa,Sporting Goods Shop
4,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Gym,Sushi Restaurant,Dessert Shop,Coffee Shop,Thai Restaurant,Movie Theater,Brewery,Café


In [94]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [95]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Check the clusters

## Cluster 1

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Hotel,Park,Department Store,Gym,Breakfast Spot,Playground,Sandwich Place,Food & Drink Shop,Donut Shop,Doner Restaurant
3,Central Toronto,0,Coffee Shop,Yoga Studio,Bagel Shop,Clothing Store,Chinese Restaurant,Café,Diner,Fast Food Restaurant,Spa,Sporting Goods Shop
4,Central Toronto,0,Pizza Place,Sandwich Place,Gym,Sushi Restaurant,Dessert Shop,Coffee Shop,Thai Restaurant,Movie Theater,Brewery,Café
5,Central Toronto,0,Trail,Restaurant,Tennis Court,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
6,Central Toronto,0,Coffee Shop,Pizza Place,Sushi Restaurant,Bagel Shop,Light Rail Station,Bank,American Restaurant,Supermarket,Vietnamese Restaurant,Eastern European Restaurant
8,Downtown Toronto,0,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Pub,Café,Chinese Restaurant,Pizza Place,Pharmacy,Sandwich Place
9,Downtown Toronto,0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Restaurant,Gay Bar,Hotel,Burrito Place,Men's Store,Mediterranean Restaurant,Gym
10,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Café,Park,Beer Store,Breakfast Spot,Sandwich Place,Restaurant,Chocolate Shop
11,Downtown Toronto,0,Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Thai Restaurant,Pizza Place,Bank,Japanese Restaurant,Cosmetics Shop
12,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Beer Bar,Restaurant,Clothing Store,Seafood Restaurant,Farmers Market,Department Store


## Cluster 2

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East YorkEast Toronto,1,Park,Convenience Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


## Cluster 3

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,2,Park,Playground,Trail,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
20,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


## Cluster 4

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Health & Beauty Service,Garden,Home Service,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


## Cluster 5

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,4,Park,Bus Line,Swim School,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
